In [ ]:
# importing packages
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline
import re
from bs4 import BeautifulSoup as bs
import requests
import string
import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords,wordnet 
from wordcloud import WordCloud
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
from time import sleep
from random import randint
import warnings
warnings.filterwarnings("ignore")


In [ ]:
hotel_linkname = 'holiday-inn-mumbai-international-airport'

In [ ]:
def scrape_reviews(hotel_linkname):
    
    info = []
    positive = []
    negative = []
    pages = np.arange(1,25)
    for page_no in pages :
        page = requests.get("https://www.booking.com/reviews/in/hotel/welcomhotel-rama-international.en-gb.html?label=gen173nr-1DCA0obEIed2VsY29taG90ZWwtcmFtYS1pbnRlcm5hdGlvbmFsSAlYBGhsiAEBmAEJuAEXyAEM2AED6AEB-AECiAIBqAIDuAL5wpuKBsACAdICJDIyM2Y3MjkyLWM3NjQtNDQxMC1hNjY2LTY5ZjM3YWIyOGMyMdgCBOACAQ;sid=cd60b337530ec521698a2d80a2891908;customer_type=total;hp_nav=0;old_page=0;order=featuredreviews;page="+ str(page_no) +";r_lang=en;rows=75&")
        soup = bs(page.text, "html.parser") # initiate a beautifulsoup object using the html source and Python’s html.parser
        review_box = soup.find("ul",{"class":"review_list"})
        sleep(randint(2,8))
        #ratings
        ratings = [i.text.strip() for i in review_box.find_all("span",{"class":"review-score-badge"})]

     #reviewer_info
        reviewer_info = [i.text.strip() for i in review_box.find_all("span",{"itemprop":"name"})]
        reviewer_name = reviewer_info[0::3]
        reviewer_country = reviewer_info[1::3]
        general_review = reviewer_info[2::3]
    # reviewer_review_times
        review_times = [i.text.strip() for i in review_box.find_all("div",{"class":"review_item_user_review_count"})]
    # review_date
        review_date = [i.text.strip().strip("Reviewed: ") for i in review_box.find_all("p",{"class":"review_item_date"})]
    # reviewer_tag
        reviewer_tag = [i.text.strip().replace("\n\n\n","").replace("•",",").lstrip(", ") for i in review_box.find_all("ul",{"class":"review_item_info_tags"})]
# positive_review
        positive_review = [i.text.strip() for i in review_box.find_all("p",{"class":"review_pos"})]
        # negative_review
        negative_review = [i.text.strip() for i in review_box.find_all("p",{"class":"review_neg"})]
        # append all reviewers’ info into one list
        for i in range(len(reviewer_name)):
            info.append([ratings[i],reviewer_name[i],reviewer_country[i],general_review[i],review_times[i],review_date[i],reviewer_tag[i]])
        # build positive review list
        for i in range(len(positive_review)):
            positive.append(positive_review[i])
        # build negative review list
        for i in range(len(negative_review)):
            negative.append(negative_review[i])
        # page change
#         page_number +=1
        #Reviewer_info df
        reviewer_info = pd.DataFrame(info,columns = ["Rating","Name","Country","Overall_review","Review_times","Review_date","Review_tags"])
        reviewer_info["Rating"] = pd.to_numeric(reviewer_info["Rating"] )
        reviewer_info["Review_times"] = pd.to_numeric(reviewer_info["Review_times"].apply(lambda x:re.findall("\d+", x)[0]))
        reviewer_info["Review_date"] = pd.to_datetime(reviewer_info["Review_date"])
 
 #positive & negative reviews dfs
        pos_reviews = pd.DataFrame(positive,columns = ["positive_reviews"])
        neg_reviews = pd.DataFrame(negative,columns = ["negative_reviews"])
        return reviewer_info, pos_reviews, neg_reviews

    
 

In [ ]:
def scrape(hotel_linkname, pages):
    url ="https://www.booking.com/reviews/in/hotel/"+ hotel_linkname +".html?page="
    info = []
    positive = []
    negative = []
    for page in range(1,pages+1):
        response = requests.get(url + str(page))
        soup = bs(response.text, 'html.parser')
        review_box = soup.find("ul",{"class":"review_list"})
        sleep(randint(2,6))
        ratings = [i.text.strip() for i in review_box.find_all("span",{"class":"review-score-badge"})]
        reviewer_info = [i.text.strip() for i in review_box.find_all("span",{"itemprop":"name"})]
        reviewer_name = reviewer_info[0::3]
        #reviewer_info
        reviewer_info = [i.text.strip() for i in review_box.find_all("span",{"itemprop":"name"})]
        reviewer_name = reviewer_info[0::3]
        reviewer_country = reviewer_info[1::3]
        general_review = reviewer_info[2::3]
        # reviewer_review_times
        review_times = [i.text.strip() for i in review_box.find_all("div",{"class":"review_item_user_review_count"})]
    # review_date
        review_date = [i.text.strip().strip("Reviewed: ") for i in review_box.find_all("p",{"class":"review_item_date"})]
    # reviewer_tag
        reviewer_tag = [i.text.strip().replace("\n\n\n","").replace("•",",").lstrip(", ") for i in review_box.find_all("ul",{"class":"review_item_info_tags"})]
    # positive_review
        positive_review = [i.text.strip() for i in review_box.find_all("p",{"class":"review_pos"})]
        # negative_review
        negative_review = [i.text.strip() for i in review_box.find_all("p",{"class":"review_neg"})]
        for i in range(len(reviewer_name)):
            info.append([ratings[i],reviewer_name[i],reviewer_country[i],general_review[i],review_times[i],review_date[i],reviewer_tag[i]])
         # build positive review list
        for i in range(len(positive_review)):
            positive.append(positive_review[i])
        # build negative review list
        for i in range(len(negative_review)):
            negative.append(negative_review[i])
    #     #Reviewer_info df
        reviewer_info = pd.DataFrame(info,columns = ["Rating","Name","Country","Overall_review","Review_times","Review_date","Review_tags"])
        reviewer_info["Rating"] = pd.to_numeric(reviewer_info["Rating"] )
        reviewer_info["Review_times"] = pd.to_numeric(reviewer_info["Review_times"].apply(lambda x:re.findall("\d+", x)[0]))
        reviewer_info["Review_date"] = pd.to_datetime(reviewer_info["Review_date"])

        #positive & negative reviews dfs
        pos_reviews = pd.DataFrame(positive,columns = ["positive_reviews"])
        neg_reviews = pd.DataFrame(negative,columns = ["negative_reviews"])
    return reviewer_info, pos_reviews, neg_reviews
#     else:
#         return "sorry we do not have enough data to scrap"








In [ ]:
reviewer_info, pos_reviews, neg_reviews = scrape(hotel_linkname, 10)


In [ ]:
reviewer_info.head()

In [ ]:
def show_data(df):
    print("The length of the dataframe is: {}".format(len(df)))
    print("Total NAs: {}".format(reviewer_info.isnull().sum().sum()))
    return df.head()

In [ ]:
show_data(reviewer_info)

In [ ]:
df2 = reviewer_info.sort_values('Review_times', ascending = False)
df2

In [ ]:
df2['Rating'].mean()

In [ ]:
#top country other than india who gave highest reviews
top10_df = df2[(df2['Rating']>5) & (df2['Country'] != 'India')].sort_values('Rating', ascending=False)
top10_df

In [ ]:
top10_list = top10_df['Country'].tolist()
top10 = reviewer_info[reviewer_info['Country'].isin(top10_list)]
fig, ax = plt.subplots()
fig.set_size_inches(20, 5)
ax = sns.violinplot(x = 'Country', 
 y = 'Rating',
 data = top10.head(10), 
 order = top10_list,
 linewidth = 2) 
plt.suptitle('Distribution of Ratings by Country') 
plt.xticks(rotation=90)

In [ ]:
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
values = [len(pos_reviews), len(neg_reviews)]
ax.pie(values, 
 labels = ["Number of Positive Reviews", "Number of Negative Reviews"],
 colors=["gold", "lightcoral"],
 shadow=True,
 startangle=90, 
 autopct="%1.2f%%")
ax.axis("equal")
plt.title("Positive Reviews Vs. Negative Reviews");

In [ ]:
#Define tag list
tag_list = ['Business','Leisure','Group','Couple','Family','friends','Solo']
#Count for each review tag
tag_counts = []
for tag in tag_list:
    counts = reviewer_info['Review_tags'].str.count(tag).sum()
    tag_counts.append(counts)


In [ ]:
#Convert to a dataframe
counts = reviewer_info['Review_tags']
counts

In [ ]:
trip_type = pd.DataFrame({'Trip Type':tag_list,'Counts':tag_counts}).sort_values('Counts',ascending = False)
trip_type

In [ ]:
#Visualize the trip type counts from Review_tags
fig = px.bar(trip_type, x='Trip Type', y='Counts', title='Review Tags Counts for each Trip Type')
fig.show()

In [ ]:
# wordnet and treebank have different tagging systems
# Create a function to define a mapping between wordnet tags and POS tags 
def get_wordnet_pos(pos_tag):
    if pos_tag.startswith("J"):
        return wordnet.ADJ
    elif pos_tag.startswith("V"):
        return wordnet.VERB
    elif pos_tag.startswith("N"):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
 
    else:
        return wordnet.NOUN # default, return wordnet tag “NOUN”
#Create a function to lemmatize tokens in the reviews
def lemmatized_tokens(text):
    text = text.lower()
    pattern = r"\b[a-zA-Z]{3,}\b"
    tokens = nltk.regexp_tokenize(text, pattern) # tokenize the text
    tagged_tokens = nltk.pos_tag(tokens) # a list of tuples (word, pos_tag)

    stop_words = stopwords.words("english")
    new_stopwords = ['hotel','everything','anything','nothing','thing','need','good','great','excellent','perfect','much','even','really'] #customize extra stop_words
    stop_words.extend(new_stopwords)
    stop_words = set(stop_words)
    wordnet_lemmatizer = WordNetLemmatizer()
 # get lemmatized tokens #call function “get_wordnet_pos”
    lemmatized_words=[wordnet_lemmatizer.lemmatize(word, get_wordnet_pos(tag)) 
 # tagged_tokens is a list of tuples (word, tag)
    for (word, tag) in tagged_tokens if word not in stop_words and word not in string.punctuation]
    return lemmatized_words

In [ ]:
#Create a function to generate wordcloud
def wordcloud(review_df, review_colname, color, title):

  
#  INPUTS:
#  reivew_df — dataframe, positive or negative reviews
#  review_colname — column name, positive or negative review
#  color — background color of worldcloud
#  title — title of the wordcloud
#  OUTPUT:
#  Wordcloud visuazliation
  
    text = review_df[review_colname].tolist()
    text_str = " ".join(lemmatized_tokens(" ".join(text))) 
    #call function “lemmatized_tokens”
    wordcloud = WordCloud(collocations = False,
    background_color = color,
    width=1600, 
    height=800, 
    margin=2,
    min_font_size=20).generate(text_str)

    plt.figure(figsize = (15, 10))
    plt.imshow(wordcloud, interpolation = "bilinear")
    plt.axis("off")
    plt.figtext(.5,.8,title,fontsize = 20, ha="center")
    plt.show() 


    
    

    

In [ ]:
neg_reviews

In [ ]:
# Wordcoulds for Positive Reviews
wordcloud(pos_reviews,"positive_reviews", "white","Positive Reviews: ")
# # WordCoulds for Negative Reviews
wordcloud(neg_reviews,"negative_reviews", "black", "Negative Reviews:")

In [ ]:
#Create a function to get the subjectivity
def subjectivity(text): 
    return TextBlob(text).sentiment.subjectivity
#Create a function to get the polarity
def polarity(text): 
    return TextBlob(text).sentiment.polarity
#Create two new columns
reviewer_info['Subjectivity'] = reviewer_info['Overall_review'].apply(subjectivity)
reviewer_info['Polarity'] = reviewer_info['Overall_review'].apply(polarity)
#################################################################################
#Create a function to compute the negative, neutral and positive analysis
def getAnalysis(score):
    if score <0:
        return 'Negative'
    elif score == 0:
        return 'Neutral'
    else:
        return 'Positive'
reviewer_info['Analysis'] = reviewer_info['Polarity'].apply(getAnalysis)
#################################################################################
# plot the polarity and subjectivity
fig = px.scatter(reviewer_info, 
 x='Polarity', 
 y='Subjectivity', 
 color = 'Analysis',
 size='Subjectivity')
#add a vertical line at x=0 for Netural Reviews
fig.update_layout(title='Sentiment Analysis',
 shapes=[dict(type= 'line',
 yref= 'paper', y0= 0, y1= 1, 
 xref= 'x', x0= 0, x1= 0)])
fig.show()